In [0]:
%pylab inline

In [0]:
import dataiku
import dataikuapi
import pandas as pd
from dataiku.core.sql import SQLExecutor2

client = dataiku.api_client()
project = client.get_default_project()
DATASET_NAME = "iris"
original_ds = dataiku.Dataset(DATASET_NAME)
original_df = original_ds.get_dataframe()
connection_name = original_ds.get_config()['params']['connection'] # nom de la connexion SQL dataiku où aller récupérer la table

In [0]:
changes_ds_name = DATASET_NAME + "_changes"
changes_ds_creator = dataikuapi.dss.dataset.DSSManagedDatasetCreationHelper(project, changes_ds_name)
changes_ds_creator.with_store_into(connection="filesystem_managed")
changes_ds_creator.create()
changes_ds = dataiku.Dataset(changes_ds_name)
changes_ds.write_schema_from_dataframe(df=original_df)

In [0]:
editable_ds_name = DATASET_NAME + "_editable"
editable_ds_creator = dataikuapi.dss.dataset.DSSManagedDatasetCreationHelper(project, editable_ds_name)
editable_ds_creator.with_store_into(connection=connection_name) # TODO: make this configurable
editable_ds_creator.create()
editable_ds = dataiku.Dataset(editable_ds_name)
editable_ds.write_with_schema(original_df)

In [0]:
recipe_creator = dataikuapi.dss.recipe.DSSRecipeCreator("CustomCode_sync-and-apply-changes", "compute_iris_editable", project)
recipe = recipe_creator.create()

In [0]:
settings = recipe.get_settings()
settings.add_input("input", "iris")
settings.add_input("changes", "iris_changes")
settings.add_output("editable", "iris_editable")
settings.raw_params["customConfig"] = {"key": "index"}
settings.save()

Alternative:

```python
recipe_creator.with_input("iris", role="input")
recipe_creator.with_input("iris_changes", role="changes")
recipe_creator.with_output("iris_editable", role="editable")
```

In [0]:
recipe = project.get_recipe("compute_iris_editable")

In [0]:
settings = recipe.get_settings()